In [1]:
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn
from others.modules import *


In [22]:
# Put on GPU
device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# select a preset of images:

imgs_1 = noisy_imgs_1[:100]/255
imgs_2 = noisy_imgs_2[:100]/255
print(imgs_1.shape)

torch.Size([100, 3, 32, 32])


In [3]:
class Model(object):
    def __init__(self):
        self.sequence = Sequential(Convolution(3, 3, kernel_size = 3, stride = 2),
                                   ReLU(),
                                   Convolution(3,3, kernel_size = 2, stride = 2),
                                   ReLU(),
                                   Upsample(3),
                                   Convolution(3,3, kernel_size = 6, stride = 1),
                                   ReLU(),
                                   Upsample(2),
                                   Sigmoid())
                                   

        self.criterion = MSE()
        self.optimizer = SGD(self.sequence, 0.005)
        
        self.nb_samples = 100
        print('Model Initialisation')

    def train(self,train_input,train_target):
    
        for n in range(self.nb_samples):
            
            print('sample: ', n)
            print('img', train_input[n:n+1].shape)
            
            predict_img = self.sequence.forward(train_input[n:n+1])
            print('optimizer zero grad')
            self.optimizer.zero_grad()
            print('prediction', predict_img.shape)
            loss = self.criterion.forward(predict_img, train_target[n:n+1])
            backward_img = self.sequence.backward(loss)
            self.optimizer.step()
        

        return backward_img
        

In [4]:
#img = torch.randn(1, 3, 32, 32)
#img_error = torch.randn(1, 3, 32, 32)

model = Model()

output = model.train(imgs_1, imgs_2)



print('out img', output.shape)



Model Initialisation
sample:  0
img torch.Size([1, 3, 32, 32])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 7, 7])
img torch.Size([1, 3, 7, 7])
x torch.Size([1, 3, 7, 7])
kernel torch.Size([3, 3])
img torch.Size([1, 3, 21, 21])
img torch.Size([1, 3, 16, 16])
img torch.Size([1, 3, 16, 16])
x torch.Size([1, 3, 16, 16])
kernel torch.Size([2, 2])
img torch.Size([1, 3, 32, 32])
optimizer zero grad
prediction torch.Size([1, 3, 32, 32])
Sequential backward
sample:  1
img torch.Size([1, 3, 32, 32])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 7, 7])
img torch.Size([1, 3, 7, 7])
x torch.Size([1, 3, 7, 7])
kernel torch.Size([1, 3, 1, 3])
img torch.Size([1, 3, 21, 21])
img torch.Size([1, 3, 16, 16])
img torch.Size([1, 3, 16, 16])
x torch.Size([1, 3, 16, 16])
kernel torch.Size([1, 2, 1, 2])
img torch.Size([1, 3, 32, 32])
o

img torch.Size([1, 3, 16, 16])
x torch.Size([1, 3, 16, 16])
kernel torch.Size([1, 2, 1, 2])
img torch.Size([1, 3, 32, 32])
optimizer zero grad
prediction torch.Size([1, 3, 32, 32])
Sequential backward
sample:  30
img torch.Size([1, 3, 32, 32])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 7, 7])
img torch.Size([1, 3, 7, 7])
x torch.Size([1, 3, 7, 7])
kernel torch.Size([1, 3, 1, 3])
img torch.Size([1, 3, 21, 21])
img torch.Size([1, 3, 16, 16])
img torch.Size([1, 3, 16, 16])
x torch.Size([1, 3, 16, 16])
kernel torch.Size([1, 2, 1, 2])
img torch.Size([1, 3, 32, 32])
optimizer zero grad
prediction torch.Size([1, 3, 32, 32])
Sequential backward
sample:  31
img torch.Size([1, 3, 32, 32])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 7, 7])
img torch.Size([1, 3, 7, 7])
x torch.Size([1, 3, 7, 7])
kernel torch.Size([1, 3,

sample:  75
img torch.Size([1, 3, 32, 32])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 7, 7])
img torch.Size([1, 3, 7, 7])
x torch.Size([1, 3, 7, 7])
kernel torch.Size([1, 3, 1, 3])
img torch.Size([1, 3, 21, 21])
img torch.Size([1, 3, 16, 16])
img torch.Size([1, 3, 16, 16])
x torch.Size([1, 3, 16, 16])
kernel torch.Size([1, 2, 1, 2])
img torch.Size([1, 3, 32, 32])
optimizer zero grad
prediction torch.Size([1, 3, 32, 32])
Sequential backward
sample:  76
img torch.Size([1, 3, 32, 32])
Sequential forward
img torch.Size([1, 3, 32, 32])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 15, 15])
img torch.Size([1, 3, 7, 7])
img torch.Size([1, 3, 7, 7])
x torch.Size([1, 3, 7, 7])
kernel torch.Size([1, 3, 1, 3])
img torch.Size([1, 3, 21, 21])
img torch.Size([1, 3, 16, 16])
img torch.Size([1, 3, 16, 16])
x torch.Size([1, 3, 16, 16])
kernel torch.Size([1, 2, 1, 2])
img torch.Size([1, 3, 32, 32])
optimize

In [5]:
def psnr ( denoised , ground_truth ) :
        # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
        mse = torch.mean (( denoised - ground_truth ) ** 2)
        return -10 * torch.log10 ( mse + 10** -8)